### 1. 交易

#### 1.1 普通交易

In [12]:
def new_transaction(_from, to, value):
    return {
        "from": _from,
        "to": to,
        "value": value
    }

new_transaction("alice", "bob", 50)

{'from': 'alice', 'to': 'bob', 'value': 50}

In [13]:
def mine():
    return new_transaction("0", "alice", 50)

mine()

{'from': '0', 'to': 'alice', 'value': 50}

In [16]:
import json
import hashlib
import base58
from pprint import pprint

def new_transaction(_from, to, value):
    trans = {
        "from": _from,
        "to": to,
        "value": value
    }
    trans["txid"] = _hash(trans)
    return trans

def _hash(data):
    # 先使用 json.dumps 将字典转换为字符串
    dump_data = json.dumps(data, sort_keys=True)
    sha_data = hashlib.sha256(dump_data.encode())
    # 使用 base58 对 sha256 算法获取的摘要进行编码，这样可以大幅缩小摘要的长度
    return base58.b58encode(sha_data.digest()).decode()

def mine():
    return new_transaction("0", "alice", 50)

to_alice = mine()
pprint(to_alice, sort_dicts=True)

{'from': '0',
 'to': 'alice',
 'txid': 'j9iPoqLEynzx3vzymUsa2BqVGn4cyT7expv4EUUjvwH',
 'value': 50}


In [18]:
to_bob = new_transaction(to_alice["txid"], "bob", 50)
pprint(to_bob, sort_dicts=True)

{'from': 'j9iPoqLEynzx3vzymUsa2BqVGn4cyT7expv4EUUjvwH',
 'to': 'bob',
 'txid': '7qWJT6BgTC5LsaWevnZkDeRJe2awXaXwMFfiRNcR7SD9',
 'value': 50}


#### 1.2 公钥与私钥

In [19]:
import base58
import ecdsa

CURVE = ecdsa.SECP256k1
# 生成公钥和私钥
def generate_ecdsa_keys():
    # https://github.com/tlsfuzzer/python-ecdsa
    _sk = ecdsa.SigningKey.generate(curve=CURVE)  # this is your sign (private key)
    vk = _sk.get_verifying_key()  # this is your verification key (public key)
    pk = base58.b58encode(vk.to_string()).decode()
    sk = base58.b58encode(_sk.to_string()).decode()
    return pk, sk

# 使用私钥签名数据
def sign_data(secret_key, to_sig_data: str):
    _sk = base58.b58decode(secret_key.encode())
    sk = ecdsa.SigningKey.from_string(_sk, curve=CURVE)
    signature = base58.b58encode(sk.sign(to_sig_data.encode())).decode()
    return signature

# alice 的公私钥
a_pk, a_sk = generate_ecdsa_keys()
print(f"a_pk = {a_pk}")
print(f"a_sk = {a_sk}")
#a_pk = '3D7f8Ku26a5LWoM9QzL4zft9QGuypCuLvgeeP4uenWmBXCHfFR3r2eb33q2oUqzBTLh3MV2Bp7wSj7XqjtSebjhh'
#a_sk = '7Wtwfzt5kyz6AqKorXqPNc9ifaLzVCAWZf5o4h2jpcSK'

a_pk = 3oAUXxAu9MNB44tCwiGC3Ck5Ge1X1szXELk4GYpXRMyJcACC95kQTa1SWsXdZWRWShHW5XBzTZ7DpJJr2nFpGCwK
a_sk = 8UzgEATHABtyCmJTD2HfSbcEEM6kf3w49wsSpEYFDD3G


In [20]:
b_pk, b_sk = generate_ecdsa_keys()
print(f"b_pk = {b_pk}")
print(f"b_sk = {b_sk}")


b_pk = 2fKiy3HdeW7zFCUTe9TqjxRzVSdVMNxECgVW9wQvoo2NtYqSRGrXStcqc6pXHLSqf7shzSCD8NfBpju2Awhns3uq
b_sk = 4UtrVJv7tiU2po6t69mnAtt275YUqcfSzEmuDEdqeCg8


In [21]:
def new_transaction(_from, to, value, sig):
    trans = {
        "from": _from,
        "sig": sig,
        "to": to,
        "value": value
    }
    trans["txid"] = _hash(trans)
    return trans

def mine(pk):
    return new_transaction("0", pk, 50, '') # 创币交易不需要签名

mine_trans = mine(a_pk)
pprint(mine_trans, sort_dicts=True)

{'from': '0',
 'sig': '',
 'to': '3oAUXxAu9MNB44tCwiGC3Ck5Ge1X1szXELk4GYpXRMyJcACC95kQTa1SWsXdZWRWShHW5XBzTZ7DpJJr2nFpGCwK',
 'txid': '3i5reCPXeurZCjq1XnsQF1g8KNWQRsCMCs8tSsVp1iW4',
 'value': 50}


In [22]:
m_txid = mine_trans["txid"]
# 使用私钥对数据进行签名
a_sig = sign_data(a_sk, m_txid)
new_transaction(m_txid, b_pk, 50, a_sig)

{'from': '3i5reCPXeurZCjq1XnsQF1g8KNWQRsCMCs8tSsVp1iW4',
 'sig': '5T5vG2nt5LpzaMvPXVsWamagrw1iKeVFjQcQxYbMPgueXtdHSSYULCTavhxVnwEEYBtXwGdoZ4i7MvBJ4noD8HYk',
 'to': '2fKiy3HdeW7zFCUTe9TqjxRzVSdVMNxECgVW9wQvoo2NtYqSRGrXStcqc6pXHLSqf7shzSCD8NfBpju2Awhns3uq',
 'value': 50,
 'txid': '2LwmGZKQaPjq2Pq7QnEFopWVZ7m2qRcEVMiHrtpiP18d'}

#### 1.3 比特币交易脚本

In [23]:
def new_transaction(_from, script_pubkey, value, sig, script_type):
    trans = {
        "from": _from,
        "sig": sig,
        "script_pubkey": script_pubkey,
        "script_type": script_type,
        "value": value
    }
    if script_type == 'p2pk':
        trans["script_pubkey"] = f'{script_pubkey} OP_CHECKSIG'
    trans["txid"] = _hash(trans)
    return trans


def mine(script_pubkey, script_type):
    return new_transaction("0", script_pubkey, 50, '', script_type)  # 创币交易不需要签名


# alice 的创币交易
mine_trans = mine(a_pk, 'p2pk')
pprint(mine_trans, sort_dicts=True)

{'from': '0',
 'script_pubkey': '3oAUXxAu9MNB44tCwiGC3Ck5Ge1X1szXELk4GYpXRMyJcACC95kQTa1SWsXdZWRWShHW5XBzTZ7DpJJr2nFpGCwK '
                  'OP_CHECKSIG',
 'script_type': 'p2pk',
 'sig': '',
 'txid': 'Ei8iiUTt2rJ1P6Aned5DZjErfWUJXVkvCsGe7GrFGkd3',
 'value': 50}


In [24]:
m_txid = mine_trans["txid"]
a_sig = sign_data(a_sk, m_txid)
new_transaction(m_txid, b_pk, 50, a_sig, 'p2pk')

{'from': 'Ei8iiUTt2rJ1P6Aned5DZjErfWUJXVkvCsGe7GrFGkd3',
 'sig': 'avgYYUqLfe9C2GNUDqxjmHi6yRXD6bKBzaemZ7EyXguTowPvwKKPDaYphYuMZmWJAJMZNdT51gBcfd4eLzzKwae',
 'script_pubkey': '2fKiy3HdeW7zFCUTe9TqjxRzVSdVMNxECgVW9wQvoo2NtYqSRGrXStcqc6pXHLSqf7shzSCD8NfBpju2Awhns3uq OP_CHECKSIG',
 'script_type': 'p2pk',
 'value': 50,
 'txid': 'Fp9PWqHZhBLFcosp1mnGmaeHVmdiKHCbP1Q54BreXfsJ'}

In [28]:
def eval_script(script: str, data: str):
    """

    :param script: 比特币脚本
    :param data: 签名的原始数据
    """
    stack = []
    # 通过空格分离脚本的各部分 (合成脚本的时候就是使用空格拼接的)
    tax_list = script.strip().split()
    for tax in tax_list:
        match tax:
            # 检测到 OP_CHECKSIG 关键词，弹出栈顶2个元素，其中第一个是公钥，第二个是签名
            case "OP_CHECKSIG":
                # <sig> <pubk> OP_CHECKSIG
                pubk = stack.pop()
                sig = stack.pop()
                # 调用函数验证签名
                if not validate_sig_data(pubk, sig, data):
                    return False
            case str() as s:
                stack.append(s)

    return True if not stack else False


# 传入公钥，签名和 签名前的数据 来验证签名是否正确。
def validate_sig_data(public_key: str, sign: str, to_sig_data: str):
    public_key = base58.b58decode(public_key.encode())
    signature = base58.b58decode(sign.encode())
    vk = ecdsa.VerifyingKey.from_string(public_key, curve=ecdsa.SECP256k1)
    try:
        return vk.verify(signature, to_sig_data.encode())
    except:
        return False

script = 'avgYYUqLfe9C2GNUDqxjmHi6yRXD6bKBzaemZ7EyXguTowPvwKKPDaYphYuMZmWJAJMZNdT51gBcfd4eLzzKwae 3oAUXxAu9MNB44tCwiGC3Ck5Ge1X1szXELk4GYpXRMyJcACC95kQTa1SWsXdZWRWShHW5XBzTZ7DpJJr2nFpGCwK OP_CHECKSIG'
data = 'Ei8iiUTt2rJ1P6Aned5DZjErfWUJXVkvCsGe7GrFGkd3'  # 创币交易的 txid
eval_script(script, data)  # True

True

#### 1.4 交易多个输入和多个输出

In [29]:
def build_one_vin(txid: str, vout: str, sig: str = None):
    """
    构建一个交易输入
    :param txid: 引用的交易id
    :param vout: 引用交易的第几个输出
    :param sig: 签名
    :return:
    """
    _in = {
        "txid": txid,
        "vout": int(vout),
    }
    if sig:
        _in["sig"] = sig
    return _in


def build_one_vout(script_pubkey, value, script_type):
    """
    构建一个交易输出
    :param addr: 哈希地址
    :param value: 数量
    :param script_type: 脚本类型
    :return:
    """
    out = {
        "script_type": script_type,
        "value": value,
    }
    if script_type == 'p2pk':
        out["script_pubkey"] = f'{script_pubkey} OP_CHECKSIG'
    return out


def new_transaction(txid_in_list: list, out_list: list):
    """
    :param txid_in_list: [(txid, vout, sig)]
    :param out_list: [(script_pubkey, value, script_type)]
    """
    vin_list = [build_one_vin(*i) for i in txid_in_list]
    vout_list = [build_one_vout(*i) for i in out_list]
    trans = {
        "vin": vin_list,
        "vout": vout_list
    }
    trans["txid"] = _hash(trans)
    return trans


def mine(script_pubkey, script_type):
    # 修改成新的调用方式
    return new_transaction([("0", 0)], [(script_pubkey, 50, script_type)])


# alice 的创币交易
m_trans = mine(a_pk, 'p2pk')
pprint(m_trans, sort_dicts=True)

{'txid': 'GCFvnxaA2pbV6YFc7rijcGHbFtn1LGYTrX6VLDYuz4Lw',
 'vin': [{'txid': '0', 'vout': 0}],
 'vout': [{'script_pubkey': '3oAUXxAu9MNB44tCwiGC3Ck5Ge1X1szXELk4GYpXRMyJcACC95kQTa1SWsXdZWRWShHW5XBzTZ7DpJJr2nFpGCwK '
                            'OP_CHECKSIG',
           'script_type': 'p2pk',
           'value': 50}]}


In [30]:
# alice 向 bob 的交易
m_txid = m_trans["txid"]
a_sig = sign_data(a_sk, m_txid)
vin = [(m_txid, 0, a_sig)]
vout = [(b_pk, 50, 'p2pk')]

# 向 bob 的交易
trans = new_transaction(vin, vout)
pprint(trans, sort_dicts=True)

{'txid': '94SwTYYamGYYyKvUQ9uj3V3nfujqmzHRVPcFYNXAfJ6H',
 'vin': [{'sig': '58b6nm6nj3brFpLZ52jN6AfKsdZeBcjkvj6jTKW6VZ44HbiLJrj9NS9F8jBf31rV1d7ofeJsiX1Et7uBoSNsgrRc',
          'txid': 'GCFvnxaA2pbV6YFc7rijcGHbFtn1LGYTrX6VLDYuz4Lw',
          'vout': 0}],
 'vout': [{'script_pubkey': '2fKiy3HdeW7zFCUTe9TqjxRzVSdVMNxECgVW9wQvoo2NtYqSRGrXStcqc6pXHLSqf7shzSCD8NfBpju2Awhns3uq '
                            'OP_CHECKSIG',
           'script_type': 'p2pk',
           'value': 50}]}


#### 1.5 UTXO

In [32]:
from collections import defaultdict

UTXO = defaultdict(dict)

def adjust_UTXO(trans):
    """
    调整 UTXO
    :param trans: 一个交易
    """
    txid = trans["txid"]
    # UTXO 中删除 vin
    for vin in trans["vin"]:
        in_txid, out = vin["txid"], vin["vout"]
        if in_txid != "0":   # 创币交易引用的 txid 是 "0"，无中生有的交易，不可能在 utxo 中
            UTXO[in_txid].pop(out)
            if not UTXO[in_txid]:
                UTXO.pop(in_txid)
    # UTXO 中增加 vout
    for idx, vout in enumerate(trans["vout"]):
        UTXO[txid][idx] = vout


def new_transaction(txid_in_list: list, out_list: list):
    vin_list = [build_one_vin(*i) for i in txid_in_list]
    vout_list = [build_one_vout(*i) for i in out_list]
    trans = {
        "vin": vin_list,
        "vout": vout_list
    }
    trans["txid"] = _hash(trans)

    # 维护 UTXO
    adjust_UTXO(trans)
    return trans

# alice 的创币交易
m_trans = mine(a_pk, 'p2pk')
pprint(m_trans, sort_dicts=True)

m_txid = m_trans["txid"]
pprint(dict(UTXO), sort_dicts=True)

{'txid': 'GCFvnxaA2pbV6YFc7rijcGHbFtn1LGYTrX6VLDYuz4Lw',
 'vin': [{'txid': '0', 'vout': 0}],
 'vout': [{'script_pubkey': '3oAUXxAu9MNB44tCwiGC3Ck5Ge1X1szXELk4GYpXRMyJcACC95kQTa1SWsXdZWRWShHW5XBzTZ7DpJJr2nFpGCwK '
                            'OP_CHECKSIG',
           'script_type': 'p2pk',
           'value': 50}]}
{'GCFvnxaA2pbV6YFc7rijcGHbFtn1LGYTrX6VLDYuz4Lw': {0: {'script_pubkey': '3oAUXxAu9MNB44tCwiGC3Ck5Ge1X1szXELk4GYpXRMyJcACC95kQTa1SWsXdZWRWShHW5XBzTZ7DpJJr2nFpGCwK '
                                                                       'OP_CHECKSIG',
                                                      'script_type': 'p2pk',
                                                      'value': 50}}}


In [33]:
# alice 向 bob 的交易
a_sig = sign_data(a_sk, m_txid)
vin = [(m_txid, 0, a_sig)]
vout = [(b_pk, 50, 'p2pk')]
r = new_transaction(vin, vout)
pprint(r, sort_dicts=True)

{'txid': '9bJyU28v9gpWwJkFtWDRb5TtqN6gmHX72B9P5VWZHGho',
 'vin': [{'sig': '2Ba9vAGmeES5ik7Rm3TDt8BThAd3WUSJU9SngAuVuQX8HDRHQCwBoWTSsfotEMGZQbCX7ntkuGD7Copx793zsZdG',
          'txid': 'GCFvnxaA2pbV6YFc7rijcGHbFtn1LGYTrX6VLDYuz4Lw',
          'vout': 0}],
 'vout': [{'script_pubkey': '2fKiy3HdeW7zFCUTe9TqjxRzVSdVMNxECgVW9wQvoo2NtYqSRGrXStcqc6pXHLSqf7shzSCD8NfBpju2Awhns3uq '
                            'OP_CHECKSIG',
           'script_type': 'p2pk',
           'value': 50}]}


In [34]:
pprint(dict(UTXO), sort_dicts=True)

{'9bJyU28v9gpWwJkFtWDRb5TtqN6gmHX72B9P5VWZHGho': {0: {'script_pubkey': '2fKiy3HdeW7zFCUTe9TqjxRzVSdVMNxECgVW9wQvoo2NtYqSRGrXStcqc6pXHLSqf7shzSCD8NfBpju2Awhns3uq '
                                                                       'OP_CHECKSIG',
                                                      'script_type': 'p2pk',
                                                      'value': 50}}}


In [37]:
# 有了 utxo, 我们可以轻松找到引用交易的输出，终于可以把验证签名的代码加入到 new_transaction 函数中了
def new_transaction(txid_in_list: list, out_list: list):
    vin_list = [build_one_vin(*i) for i in txid_in_list]
    vout_list = [build_one_vout(*i) for i in out_list]
    # 验证输入列表，错误直接抛出异常
    is_valid_vin_list(vin_list)

    trans = {
        "vin": vin_list,
        "vout": vout_list
    }
    trans["txid"] = _hash(trans)

    # 维护 UTXO
    adjust_UTXO(trans)
    return trans

def is_valid_vin_list(vin_list):
    if is_mine_trans(vin_list):
        # 创币交易，跳过
        return True

    for vin in vin_list:
        txid, out_idx = vin["txid"], vin["vout"]
        if txid not in UTXO:
            raise ValueError(f"txid {txid} 无效")
        if out_idx not in UTXO[txid]:
            raise ValueError(f"txid {txid} 使用的 vout {out_idx} 无效")

        # 验证解锁脚本
        out_data = UTXO[txid][out_idx]
        if not eval_script(
            f'{vin["sig"]} {out_data["script_pubkey"]}',
            vin["txid"],
        ):
            raise ValueError(f"{txid} script result False")

def is_mine_trans(vin_list):
    return len(vin_list) == 1 and vin_list[0]["txid"] == "0"

# 清空 UTXO
UTXO = defaultdict(dict)
# alice 的创币交易
m_trans = mine(a_pk, 'p2pk')
m_txid = m_trans["txid"]

a_sig = sign_data(a_sk, m_txid)
vin = [(m_txid, 0, a_sig)]
vout = [(b_pk, 50, 'p2pk')]
b_trans = new_transaction(vin, vout)
b_trans

{'vin': [{'txid': 'GCFvnxaA2pbV6YFc7rijcGHbFtn1LGYTrX6VLDYuz4Lw',
   'vout': 0,
   'sig': '2znw9WCAnDMTiMPZgj5QuR5J495m1WsEMedVj545cD7rcCcz4jfteWCRS59dKd9uBUdNHQnEsZMxcJ94T8Efq5F3'}],
 'vout': [{'script_type': 'p2pk',
   'value': 50,
   'script_pubkey': '2fKiy3HdeW7zFCUTe9TqjxRzVSdVMNxECgVW9wQvoo2NtYqSRGrXStcqc6pXHLSqf7shzSCD8NfBpju2Awhns3uq OP_CHECKSIG'}],
 'txid': 'b2CNmBhgXn7PyzziusKDZzykmpke8dZ5MzUDQP3vp7Z'}

### 2. 区块

In [38]:
def new_block(trans_list):
    """创建区块，传入交易列表"""
    return {"transactions": trans_list, "index": _hash(trans_list)}

# 将两笔交易打包成一个区块
node_trans_list = [m_trans, b_trans]
b = new_block(node_trans_list)
pprint(b, sort_dicts=True)

{'index': '6QSk3GkN1Ya5ssw5wshWMr7j2fEYiMeLksLdQGjqMAJe',
 'transactions': [{'txid': 'GCFvnxaA2pbV6YFc7rijcGHbFtn1LGYTrX6VLDYuz4Lw',
                   'vin': [{'txid': '0', 'vout': 0}],
                   'vout': [{'script_pubkey': '3oAUXxAu9MNB44tCwiGC3Ck5Ge1X1szXELk4GYpXRMyJcACC95kQTa1SWsXdZWRWShHW5XBzTZ7DpJJr2nFpGCwK '
                                              'OP_CHECKSIG',
                             'script_type': 'p2pk',
                             'value': 50}]},
                  {'txid': 'b2CNmBhgXn7PyzziusKDZzykmpke8dZ5MzUDQP3vp7Z',
                   'vin': [{'sig': '2znw9WCAnDMTiMPZgj5QuR5J495m1WsEMedVj545cD7rcCcz4jfteWCRS59dKd9uBUdNHQnEsZMxcJ94T8Efq5F3',
                            'txid': 'GCFvnxaA2pbV6YFc7rijcGHbFtn1LGYTrX6VLDYuz4Lw',
                            'vout': 0}],
                   'vout': [{'script_pubkey': '2fKiy3HdeW7zFCUTe9TqjxRzVSdVMNxECgVW9wQvoo2NtYqSRGrXStcqc6pXHLSqf7shzSCD8NfBpju2Awhns3uq '
                                              'OP_CHE

#### 2.1 哈希链接

In [39]:
BlockChain = []

# 我们将上面2个交易拆成2个区块演示一下
# 假设 alice 的创币交易就是创世区块，此时 prvious_hash 为 ‘’
def new_block(trans_list):
    prv_hash = BlockChain[-1]["index"] if BlockChain else ''
    block = {"transactions": trans_list, "previous_hash": prv_hash}
    block["index"] = _hash(block)
    BlockChain.append(block)
    return block

new_block([m_trans])
pprint(BlockChain, sort_dicts=True)

[{'index': '4hccSDZepn4jvM3XiD8F31sCuKgXcYmSoaj7PGQqJjcQ',
  'previous_hash': '',
  'transactions': [{'txid': 'GCFvnxaA2pbV6YFc7rijcGHbFtn1LGYTrX6VLDYuz4Lw',
                    'vin': [{'txid': '0', 'vout': 0}],
                    'vout': [{'script_pubkey': '3oAUXxAu9MNB44tCwiGC3Ck5Ge1X1szXELk4GYpXRMyJcACC95kQTa1SWsXdZWRWShHW5XBzTZ7DpJJr2nFpGCwK '
                                               'OP_CHECKSIG',
                              'script_type': 'p2pk',
                              'value': 50}]}]}]


In [40]:
new_block([b_trans])
pprint(BlockChain, sort_dicts=True)

[{'index': '4hccSDZepn4jvM3XiD8F31sCuKgXcYmSoaj7PGQqJjcQ',
  'previous_hash': '',
  'transactions': [{'txid': 'GCFvnxaA2pbV6YFc7rijcGHbFtn1LGYTrX6VLDYuz4Lw',
                    'vin': [{'txid': '0', 'vout': 0}],
                    'vout': [{'script_pubkey': '3oAUXxAu9MNB44tCwiGC3Ck5Ge1X1szXELk4GYpXRMyJcACC95kQTa1SWsXdZWRWShHW5XBzTZ7DpJJr2nFpGCwK '
                                               'OP_CHECKSIG',
                              'script_type': 'p2pk',
                              'value': 50}]}]},
 {'index': '3wbwouFDALTwA2bzRN65W32DMdiRMLk99HJo5ETJkBw5',
  'previous_hash': '4hccSDZepn4jvM3XiD8F31sCuKgXcYmSoaj7PGQqJjcQ',
  'transactions': [{'txid': 'b2CNmBhgXn7PyzziusKDZzykmpke8dZ5MzUDQP3vp7Z',
                    'vin': [{'sig': '2znw9WCAnDMTiMPZgj5QuR5J495m1WsEMedVj545cD7rcCcz4jfteWCRS59dKd9uBUdNHQnEsZMxcJ94T8Efq5F3',
                             'txid': 'GCFvnxaA2pbV6YFc7rijcGHbFtn1LGYTrX6VLDYuz4Lw',
                             'vout': 0}],
                    'vout': [

#### 2.2 工作量证明算法

In [44]:
def proof_of_work(block: dict):
    """
    简单的工作量证明算法
    找到一个数，使得区块的hash前4位为0
    :param block: 区块
    :return:
    """
    # 工作量证明--->穷举法计算出特殊的数
    block_hash = _hash(block)
    proof = 0
    while True:
        ret, hash_val = valid_proof(proof, block_hash)
        if ret:
            return proof, hash_val
        proof += 1


def valid_proof(proof, block_hash) -> (bool, str):
    """
    验证工作量证明，计算出的hash是否正确
    对上一个区块的proof和hash与当期区块的proof做sha256运算、
    :param proof: 当前区块的随机数（工作量）
    :param block_hash: 本区块的 hash
    """
    guess = f"{proof}{block_hash}"
    guess_hash = hashlib.sha256(guess.encode()).hexdigest()
    return guess_hash[:4] == "0000", guess_hash

def new_block(trans_list):
    prv_hash = BlockChain[-1]["index"] if BlockChain else ''
    block = {"transactions": trans_list, "previous_hash": prv_hash}
    block["proof"], block["index"] = proof_of_work(block)
    BlockChain.append(block)
    return block


block = new_block([m_trans])
pprint(block, sort_dicts=True)

{'index': '00008b7b00863ed748bd53fb4d24a7d1243d1f8a95b401ff0167b088294157ec',
 'previous_hash': '0000004b13c13d0b697696f536cc06ed021404f01b24a944ccea1b5c38875d44',
 'proof': 22047,
 'transactions': [{'txid': 'GCFvnxaA2pbV6YFc7rijcGHbFtn1LGYTrX6VLDYuz4Lw',
                   'vin': [{'txid': '0', 'vout': 0}],
                   'vout': [{'script_pubkey': '3oAUXxAu9MNB44tCwiGC3Ck5Ge1X1szXELk4GYpXRMyJcACC95kQTa1SWsXdZWRWShHW5XBzTZ7DpJJr2nFpGCwK '
                                              'OP_CHECKSIG',
                             'script_type': 'p2pk',
                             'value': 50}]}]}


In [46]:
import time
BlockChain = []
def new_block(trans_list):
    prv_hash = BlockChain[-1]["index"] if BlockChain else ''
    block = {
        "transactions": trans_list,
        "previous_hash": prv_hash,
        # 高度
        "height": len(BlockChain),
        # 时间戳
        "timestamp": time.time(),
    }
    block["proof"], block["index"] = proof_of_work(block)
    BlockChain.append(block)
    return block

block = new_block([m_trans])
pprint(block, sort_dicts=True)

{'height': 0,
 'index': '000048eb5aa1e6126f320883104fd45cd4ae9798321cac9068996a66faec5688',
 'previous_hash': '',
 'proof': 37444,
 'timestamp': 1660037368.8697972,
 'transactions': [{'txid': 'GCFvnxaA2pbV6YFc7rijcGHbFtn1LGYTrX6VLDYuz4Lw',
                   'vin': [{'txid': '0', 'vout': 0}],
                   'vout': [{'script_pubkey': '3oAUXxAu9MNB44tCwiGC3Ck5Ge1X1szXELk4GYpXRMyJcACC95kQTa1SWsXdZWRWShHW5XBzTZ7DpJJr2nFpGCwK '
                                              'OP_CHECKSIG',
                             'script_type': 'p2pk',
                             'value': 50}]}]}


In [11]:

def hash_256(s, b58=True):
    data = hashlib.sha256(s.encode())
    if b58:
        return base58.b58encode(data.digest()).decode()
    else:
        return data.hexdigest()

h_b_pk = hash_256(b_pk)
h_b_pk

'3YX4udWpjb7U2vTDwdMkz36y65cVroq1mnwnpy8SAkkw'